In [151]:
import pandas as pd

df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

c:\Users\Gabri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [152]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import catboost

In [92]:
idk = df

In [162]:
df['amenities'] = df['amenities'].apply(lambda x: x.strip('{').strip('}').replace('"', '').split(','))

In [163]:
amenities = {value for row in df['amenities'] for value in row}

In [164]:
amenities = list(amenities)

In [169]:
# df_exploded1 = df.explode('amenities')

# one_hot_encoded1 = pd.get_dummies(df_exploded1['amenities'], prefix='amenities')

# df_encoded1 = pd.concat([df_exploded1, one_hot_encoded1], axis=1)

# df_aggregated1 = df_encoded1.groupby(level=0).sum()

# amenities_cleaned = pd.concat([df, df_aggregated1], axis=1)

In [167]:
# amenities_cleaned.head()

In [117]:
# for i in range(len(amenities)):
#     idk[amenities[i]] = idk['amenities'].apply(lambda x: x[i])
#     test_df[amenities[i]] = test_df['amenities'].apply(lambda x: x[i])

# test_df['amenities']

In [174]:
df['amenities_j'] = df['amenities'].apply(lambda x: ','.join(x))

In [173]:
from sklearn.feature_extraction.text import CountVectorizer
from catboost import CatBoostRegressor
from scipy.sparse import hstack

In [175]:
vectorizer = CountVectorizer()
x_tags = vectorizer.fit_transform(df['amenities_j'])

In [153]:
def percentage_to_float(percentage):
    if type(percentage) == str:
        stripped = percentage.strip('%')
        value = float(stripped) / 100 
        return value

df['host_response_rate'] = df['host_response_rate'].apply(lambda x: percentage_to_float(x))

df['extra_people'] = df['extra_people'].apply(lambda x: x.strip('$'))

df['host_listings_count'] = df['host_listings_count'].fillna(0).astype(int)

df['bedrooms'] = df['bedrooms'].fillna(0).astype(int)

df['beds'] = df['beds'].fillna(0).astype(int)

df['square_feet'] = df['square_feet'].fillna(0).astype(float)

df['review_scores_rating'] = df['review_scores_rating'].fillna(0).astype(int)

df['review_scores_cleanliness'] = df['review_scores_cleanliness'].fillna(0).astype(int)

df['review_scores_checkin'] = df['review_scores_checkin'].fillna(0).astype(int)

df['review_scores_communication'] = df['review_scores_communication'].fillna(0).astype(int)

df['review_scores_location'] = df['review_scores_location'].fillna(0).astype(int)

df['review_scores_value'] = df['review_scores_value'].fillna(0).astype(int)

# df['host_is_superhost'] = df['host_is_superhost'].replace({'f': 0, 't': 1})

# df['host_identity_verified'] = df['host_identity_verified'].replace({'f': 0, 't': 1})

# df['instant_bookable'] = df['instant_bookable'].replace({'f': 0, 't': 1})

# df['require_guest_profile_picture'] = df['require_guest_profile_picture'].replace({'f': 0, 't': 1})

# df['require_guest_phone_verification'] = df['require_guest_phone_verification'].replace({'f': 0, 't': 1})


In [177]:
from math import isnan

def to_float(s):
    try:
        ret = float(s)
    except:
        ret = -1
    if isnan(ret):
        ret = -1
    return ret

def to_int(s):
    try:
        ret = int(s)
    except:
        ret = -1
    return ret

categories = [x for x in list(set(df['neighbourhood_group_cleansed'])) if type(x) is str]

def create_feature(row):
    guest_num = to_int(row.guests_included)
    guest_cap = to_int(row.extra_people)
    bedrooms = to_int(row.bedrooms)
    bathrooms = to_int(row.bathrooms)
    review = to_float(row.review_scores_rating)
    rate = to_float(row.host_response_rate)
    extra = to_float(row.extra_people)
    list_count = to_int(row.host_listings_count)
    bathrooms = to_float(row.bathrooms)
    bedrooms = to_int(row.bedrooms)
    beds = to_int(row.beds)
    accommodates = to_int(row.accommodates)
    sq_feet = to_float(row.square_feet)
    guests = to_int(row.guests_included)
    min_nights = to_int(row.minimum_nights)
    max_nights = to_int(row.maximum_nights)
    rev_rating = to_int(row.review_scores_rating)
    clean = to_int(row.review_scores_cleanliness)
    check_in = to_int(row.review_scores_checkin)
    comm = to_int(row.review_scores_communication)
    location = to_int(row.review_scores_location)
    rev_value = to_int(row.review_scores_value)
    # superhost = to_float(row.host_is_superhost)
    # host_ver = to_int(row.host_identity_verified)
    # instant = to_int(row.instant_bookable)
    # guest_pp = to_int(row.require_guest_profile_picture)
    # guest_ver = to_int(row.require_guest_phone_verification)
    one_hot = [int(row.neighbourhood_group_cleansed == category) for category in categories]
    
    return [guest_num, guest_cap, bedrooms, bathrooms, review, rate, extra, list_count, bathrooms, \
            bedrooms, beds, accommodates, sq_feet, guests, min_nights, max_nights, rev_rating, clean, \
                check_in, comm, location, rev_value] + one_hot

train_X, train_y = [], []
for (idx, row) in df.iterrows():
    price = float(row.price)
    feature = create_feature(row)

    train_X.append(feature)
    train_y.append(price)

#print(len(train_X), len(train_y))

In [178]:
hstack(x_tags, train_X)

ValueError: blocks must be 2-D

In [140]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(train_X, train_y)

LinearRegression()

In [7]:
import pandas as pd

test_df = pd.read_csv('test.csv')

c:\Users\Gabri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [159]:
model = catboost.CatBoostRegressor()  
model.fit(train_X, train_y)

Learning rate set to 0.071321
0:	learn: 129.4903019	total: 3.97ms	remaining: 3.97s
1:	learn: 126.3220186	total: 7.46ms	remaining: 3.72s
2:	learn: 123.4133843	total: 10.9ms	remaining: 3.62s
3:	learn: 120.8103031	total: 14.4ms	remaining: 3.57s
4:	learn: 118.5227802	total: 18.9ms	remaining: 3.76s
5:	learn: 116.5275717	total: 23.7ms	remaining: 3.93s
6:	learn: 114.7563389	total: 27.8ms	remaining: 3.95s
7:	learn: 113.1057185	total: 31.6ms	remaining: 3.92s
8:	learn: 111.6710861	total: 35.5ms	remaining: 3.91s
9:	learn: 110.3894539	total: 39.7ms	remaining: 3.93s
10:	learn: 109.1524582	total: 43.6ms	remaining: 3.92s
11:	learn: 108.0291749	total: 47.3ms	remaining: 3.9s
12:	learn: 107.1192792	total: 51.5ms	remaining: 3.91s
13:	learn: 106.1404628	total: 55.2ms	remaining: 3.89s
14:	learn: 105.2481661	total: 59.4ms	remaining: 3.9s
15:	learn: 104.5415038	total: 63.2ms	remaining: 3.88s
16:	learn: 103.7910007	total: 66.9ms	remaining: 3.87s
17:	learn: 103.1962564	total: 70.5ms	remaining: 3.85s
18:	learn:

In [160]:
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
    feature = create_feature(row)
    test_ids.append(row.id)
    test_X.append(feature)
test_y = model.predict(test_X)
#test_y = regr.predict(test_X)

In [147]:
# rf_class = RandomForestClassifier(random_state=42)

# rf_class.fit(train_X, train_y)

# test_y = rf_class.predict(test_X)

In [9]:
print(len(test_y))

17337


In [161]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
output_df.to_csv('simple_linear_regression_baseline.csv', index = False)

In [11]:
import numpy as np

mean = np.mean(list(df['price']))

In [12]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = [mean for i in range(len(test_ids))]
output_df.to_csv('mean_value_baseline.csv', index = False)

In [35]:
data1 = [[0,1,2,3,4,5,6,7],
         ['a','b','c','d','e','f','g','h']]

data2 = [[0,1,2,3,4,5,6,7],
        ['ab','bc','cd','dh','ej','fe','gs','ah']]

df_df1 = pd.DataFrame(data1) 
df_df2 = pd.DataFrame(data2) 



df_df_j = pd.concat([df_df1, df_df2])

In [36]:
df_df1.head()

,0,1,2,3,4,5,6,7
0,0,1,2,3,4,5,6,7
1,a,b,c,d,e,f,g,h


In [37]:
df_df2.head()

,0,1,2,3,4,5,6,7
0,0,1,2,3,4,5,6,7
1,ab,bc,cd,dh,ej,fe,gs,ah


In [38]:
df_df_j.head()

,0,1,2,3,4,5,6,7
0,0,1,2,3,4,5,6,7
1,a,b,c,d,e,f,g,h
0,0,1,2,3,4,5,6,7
1,ab,bc,cd,dh,ej,fe,gs,ah
